# Playground

## Load

In [1]:
from __future__ import annotations
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm
import theano.tensor as tt
import arviz as az

In [2]:
from m5.data import calendar, sell_prices, sales_train_eval, sales_train_valid, sample_submission